Analysis to understand correlation between PGE's hourly demand, demand forecast, and daytime mean temperature. 


Also understand the accuracy of PGE's demand forecast. 

Background: PGE is the major 